In [1]:
pip install pandas requests yfinance pyarrow fastparquet

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --------------------------- ------------ 2.1/3.0 MB 11.8 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 11.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.5 MB 11.2 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.5 MB 11.6 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.5 MB 11.6 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.5 MB 11.7 MB

# **Data Loading**

## *US GDP*

In [4]:
import pandas as pd

# Assuming US_GDP.csv path
us_gdp_csv_path = r'D:\PyTorch Projects\Malaysia_GDP\Datasets\US_GDP_Change.csv'

# Ingest US GDP data (Annual Growth Rate)
print("\n--- Ingesting US GDP data ---")
US_GDP = pd.read_csv(us_gdp_csv_path)

# Convert 'DATE' to datetime
US_GDP['observation_date'] = pd.to_datetime(US_GDP['observation_date'])

# Set 'DATE' as index
US_GDP = US_GDP.set_index('observation_date')

# Rename the index to 'date' for consistency
US_GDP.index.name = 'date'

# Rename the 'GDP' column to 'us_gdp_growth'
US_GDP = US_GDP.rename(columns={'GDP_PCH': 'us_gdp_growth'})

# Ensure the index is sorted in ascending order
US_GDP = US_GDP.sort_index(ascending=True)

# *** IMPORTANT FIX: Convert quarter-start dates to quarter-end dates ***
# This converts the index to PeriodIndex (e.g., 2007Q1), then back to Timestamp at the end of the period,
# and finally normalizes to midnight.
US_GDP.index = US_GDP.index.to_period('Q').to_timestamp(how='end').normalize()

# Create df_us_gdp_growth_q as a copy for consistency with other quarterly variables
df_us_gdp_growth_q = US_GDP.copy()

print("US GDP data loaded and processed successfully.")
print(df_us_gdp_growth_q.head())
print("======================================")
print(df_us_gdp_growth_q.info())


--- Ingesting US GDP data ---
US GDP data loaded and processed successfully.
            us_gdp_growth
date                     
1947-06-30        1.15313
1947-09-30        1.47052
1947-12-31        4.07076
1948-03-31        2.30880
1948-06-30        2.56828
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 312 entries, 1947-06-30 to 2025-03-31
Freq: QE-DEC
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   us_gdp_growth  312 non-null    float64
dtypes: float64(1)
memory usage: 4.9 KB
None


## *Straits Time Index (STI) Data*

In [12]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import yfinance as yf

# Ingesting STI data using yfinance 
print("\n--- Ingesting STI data using yfinance ---")
sti = yf.download("^STI", start="1990-01-01", end="2025-06-01", interval="1mo")

sti = sti.rename(columns={'Close': 'sti_close'})

if isinstance(sti.columns, pd.MultiIndex):
    df_sti = sti.xs('sti_close', level=0, axis=1, drop_level=False)
    df_sti.columns = ['sti_close'] 
else:
    df_sti = sti[['sti_close']]

# Set 'Date' (which is currently the index) to lowercase 'date'
df_sti.index.name = 'date' 

# Ensure the index is sorted in ascending order
df_sti = df_sti.sort_index(ascending=True)


print("STI data loaded successfully.")
print(df_sti.head())
print("======================================")
print(df_sti.info())

# Resampling STI (take average of quarter)
df_sti_q = df_sti['sti_close'].resample('Q').mean().to_frame()

print(df_sti_q.head())
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") 
print(df_sti_q.info())

[*********************100%***********************]  1 of 1 completed


--- Ingesting STI data using yfinance ---
STI data loaded successfully.
              sti_close
date                   
1990-01-01  1515.000000
1990-02-01  1550.099976
1990-03-01  1581.099976
1990-04-01  1458.400024
1990-05-01  1553.599976
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 425 entries, 1990-01-01 to 2025-05-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sti_close  425 non-null    float64
dtypes: float64(1)
memory usage: 6.6 KB
None
              sti_close
date                   
1990-03-31  1548.733317
1990-06-30  1513.000000
1990-09-30  1310.466675
1990-12-31  1138.466675
1991-03-31  1405.900024
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 142 entries, 1990-03-31 to 2025-06-30
Freq: QE-DEC
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sti_close  142 non-null    float64
dtypes: 


C:\Users\Travi\AppData\Local\Temp\ipykernel_210220\1766820323.py:29: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_sti_q = df_sti['sti_close'].resample('Q').mean().to_frame()


## *Singapore Consumer Price Index (CPI)*

In [55]:
import pandas as pd

df_sg_cpi = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\Singapore_CPI.csv")

# For mapping data to similar format/index
quarter_map = {
    "1Q": "-03-31",
    "2Q": "-06-30",
    "3Q": "-09-30",
    "4Q": "-12-31"
}

# Replace quarter with end-of-quarter date
df_sg_cpi["date"] = df_sg_cpi["Data Series"].str.replace(
    r"\s(\dQ)", 
    lambda x: quarter_map[x.group(1)], 
    regex=True
)

# Convert to datetime & set as index
df_sg_cpi["date"] = pd.to_datetime(df_sg_cpi["date"])
df_sg_cpi.set_index("date", inplace=True)

# Step 7: Drop original 'Data Series' column & sort ascendingly
df_sg_cpi.drop(columns=["Data Series"], inplace=True)
df_sg_cpi.sort_index(inplace=True)
df_sg_cpi.rename(columns={"CPI": "sg_cpi"}, inplace=True)


print(df_sg_cpi.head())
print("======================================")
print(df_sg_cpi.info())


            sg_cpi
date              
1961-03-31    21.1
1961-06-30    20.7
1961-09-30    21.0
1961-12-31    21.0
1962-03-31    21.1
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 257 entries, 1961-03-31 to 2025-03-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sg_cpi  257 non-null    float64
dtypes: float64(1)
memory usage: 4.0 KB
None


## *Index of Industrial Protection (IIP)*

In [54]:
df_iip = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\IndexOfIndustrialProtection.csv")
df_iip.set_index("Data Series", inplace=True)
df_iip.index = df_iip.index.str.strip()
df_iip.index = pd.to_datetime(df_iip.index, format= '%Y %b')
df_iip.sort_index(inplace=True)

df_iip = df_iip.resample('Q').mean()
df_iip.index.name = 'date'
df_iip = df_iip.rename(columns={'Total (Index)': 'iip'})

print(df_iip.head())
print("======================================") 
print(df_iip.info())

                  iip
date                 
1983-03-31  11.100000
1983-06-30  10.633333
1983-09-30  11.900000
1983-12-31  12.400000
1984-03-31  12.400000
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 169 entries, 1983-03-31 to 2025-03-31
Freq: QE-DEC
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   iip     169 non-null    float64
dtypes: float64(1)
memory usage: 2.6 KB
None


C:\Users\Travi\AppData\Local\Temp\ipykernel_210220\2690421329.py:7: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_iip = df_iip.resample('Q').mean()


## *Retail Sales Index (RSI)*

In [ ]:
import pandas as pd

# Load and transpose
df_rsi = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\Retail_Sales_Index.csv", index_col=0)
df_rsi = df_rsi.T
df_rsi.columns = ['rsi']

# Map "1Q" to quarter-end date
quarter_map = {
    "1Q": "-03-31",
    "2Q": "-06-30",
    "3Q": "-09-30",
    "4Q": "-12-31"
}

df_rsi.index = df_rsi.index.str.replace(
    r'(\d{4}) (\dQ)',
    lambda x: x.group(1) + quarter_map[x.group(2)],
    regex=True
)
df_rsi.index = pd.to_datetime(df_rsi.index)
df_rsi.sort_index(inplace=True)

# Confirm structure
print(df_rsi.tail())
print("======================================")
print(df_rsi.info())

                rsi
2024-03-31   96.721
2024-06-30   91.913
2024-09-30   94.232
2024-12-31  100.985
2025-03-31   97.089
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 161 entries, 1985-03-31 to 2025-03-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rsi     161 non-null    float64
dtypes: float64(1)
memory usage: 2.5 KB
None


## *Non-Oil Domestic Exports (NODX)*

In [ ]:
import pandas as pd

# Transpose the NODX data
df_nodx_raw = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\NonOil_DomesticExports.csv", index_col=0)
df_nodx_transposed = df_nodx_raw.T  # Now rows are time, columns are categories
df_nodx_transposed.columns = ['Total Electronic Products', 'Total Non-Electronic Products']

# Parse time labels like '2024 Jan' into datetime
df_nodx_transposed.index = df_nodx_transposed.index.str.strip()
df_nodx_transposed.index = pd.to_datetime(df_nodx_transposed.index, format='%Y %b')

# Sort index chronologically
df_nodx_transposed.sort_index(inplace=True)

# Aggregate export data to quarterly frequency
df_nodx_quarterly = df_nodx_transposed.resample('Q').sum()  

# Aggregate into total NODX
df_nodx_quarterly["total_nodx"] = (
    df_nodx_quarterly["Total Electronic Products"] + df_nodx_quarterly["Total Non-Electronic Products"]
)

# Drop the individual product columns
df_nodx_quarterly = df_nodx_quarterly.drop(columns=["Total Electronic Products", "Total Non-Electronic Products"])
df_nodx_quarterly.index.name = 'date'

print(df_nodx_quarterly.head())
print("======================================")
print(df_nodx_quarterly.info())

            total_nodx
date                  
1997-03-31  20542470.0
1997-06-30  22565660.0
1997-09-30  23597897.0
1997-12-31  24917677.0
1998-03-31  22750082.0
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 113 entries, 1997-03-31 to 2025-03-31
Freq: QE-DEC
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_nodx  113 non-null    float64
dtypes: float64(1)
memory usage: 1.8 KB
None


C:\Users\Travi\AppData\Local\Temp\ipykernel_210220\4117029578.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_nodx_quarterly = df_nodx_transposed.resample('Q').sum()


## *Singapore GDP (SGD Million)* 

In [ ]:
import pandas as pd

# Transpose the data
df_sg_gdp = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\Singapore_GDP.csv", index_col=0)
df_sg_gdp = df_sg_gdp.T  # Transpose so dates are rows
df_sg_gdp.columns = ['sg_gdp']  

# Strip whitespace from index labels
df_sg_gdp.index = df_sg_gdp.index.str.strip() 

quarter_map = {
    "1Q": "-03-31",
    "2Q": "-06-30",
    "3Q": "-09-30",
    "4Q": "-12-31"
}

# Replace quarter labels with dates
df_sg_gdp['date'] = df_sg_gdp.index.str.replace(
    r'(\d{4}) (\dQ)', 
    lambda x: x.group(1) + quarter_map[x.group(2)], 
    regex=True
)

# Convert to datetime and set as index
df_sg_gdp['date'] = pd.to_datetime(df_sg_gdp['date'])
df_sg_gdp.set_index('date', inplace=True)
df_sg_gdp.sort_index(inplace=True)


print(df_sg_gdp.head())
print("===================================")
print(df_sg_gdp.info())


            sg_gdp
date              
1975-03-31  7546.0
1975-06-30  7681.5
1975-09-30  7836.6
1975-12-31  7948.1
1976-03-31  8169.7
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 201 entries, 1975-03-31 to 2025-03-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sg_gdp  201 non-null    float64
dtypes: float64(1)
memory usage: 3.1 KB
None


## *Tourist Arrivals*


In [35]:
import pandas as pd

# Transpose the tourist data
df_tourist = pd.read_csv(r"D:\PyTorch Projects\Malaysia_GDP\Datasets\TouristArrivals.csv", index_col=0)
df_tourist = df_tourist.T  # Now rows are time, columns are categories
df_tourist.columns = ['total_tourists']

# Parse time labels like '2024 Jan' into datetime
df_tourist.index = df_tourist.index.str.strip()
df_tourist.index = pd.to_datetime(df_tourist.index, format='%Y %b')

# Sort index chronologically
df_tourist.sort_index(inplace=True)

# Aggregate export data to quarterly frequency
df_tourist = df_tourist.resample('Q').sum()  
df_tourist.index.name = 'date'

print(df_tourist.head())
print("======================================")
print(df_tourist.info())

            total_tourists
date                      
1978-03-31          495822
1978-06-30          495971
1978-09-30          520251
1978-12-31          531119
1979-03-31          534810
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 189 entries, 1978-03-31 to 2025-03-31
Freq: QE-DEC
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   total_tourists  189 non-null    int64
dtypes: int64(1)
memory usage: 3.0 KB
None


C:\Users\Travi\AppData\Local\Temp\ipykernel_210220\3744971594.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_tourist = df_tourist.resample('Q').sum()


## Hong Kong GDP Growth QoQ

In [53]:
df_hk_gdp = pd.read_csv(r'D:\PyTorch Projects\Malaysia_GDP\Datasets\HK_GDP_Growth.csv')

# Map quarters to end-of-quarter dates
quarter_end_map = {
    "Q1": "-03-31",
    "Q2": "-06-30",
    "Q3": "-09-30",
    "Q4": "-12-31"
}

# Create a 'date' column
df_hk_gdp['date'] = df_hk_gdp['Year'].astype(str) + df_hk_gdp['Quarter'].map(quarter_end_map)
df_hk_gdp['date'] = pd.to_datetime(df_hk_gdp['date'])

# Set date as index and sort
df_hk_gdp.set_index('date', inplace=True)
df_hk_gdp.sort_index(inplace=True)

# Drop the redundant columns
df_hk_gdp.drop(columns=['Year', 'Quarter'], inplace=True)
df_hk_gdp.rename(columns={'GrowthRate': 'hk_gdp_growth'}, inplace=True)

print(df_hk_gdp.head())
print("======================================")
print(df_hk_gdp.info())

            hk_gdp_growth
date                     
1990-03-31            1.2
1990-06-30            2.1
1990-09-30            1.1
1990-12-31            1.0
1991-03-31            1.6
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 141 entries, 1990-03-31 to 2025-03-31
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hk_gdp_growth  141 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB
None


## Merging Into Single Dataset
* After resampling most of the data to reflect QoQ change, they are merged into a whole dataset

In [56]:
df_merged = df_sg_gdp

dfs = [df_hk_gdp, df_us_gdp_growth_q, df_sti_q, df_sg_cpi,
       df_iip, df_rsi, df_tourist, df_nodx_quarterly]

for df in dfs:
    df_merged = df_merged.merge(df, how='outer', left_index=True, right_index=True)


In [57]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 313 entries, 1947-06-30 to 2025-06-30
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sg_gdp          201 non-null    float64
 1   hk_gdp_growth   141 non-null    float64
 2   us_gdp_growth   312 non-null    float64
 3   sti_close       142 non-null    float64
 4   sg_cpi          257 non-null    float64
 5   iip             169 non-null    float64
 6   rsi             161 non-null    float64
 7   total_tourists  189 non-null    float64
 8   total_nodx      113 non-null    float64
dtypes: float64(9)
memory usage: 24.5 KB


**Saving Data Frame into a CSV file**

In [58]:
df_merged.to_csv("Combined.csv")

In [59]:
df_merged.describe()

,sg_gdp,hk_gdp_growth,us_gdp_growth,sti_close,sg_cpi,iip,rsi,total_tourists,total_nodx
count,201.000000,141.000000,312.000000,142.000000,257.000000,169.000000,161.000000,1.890000e+02,1.130000e+02
mean,57436.607960,0.777305,1.563294,2496.396079,56.080934,57.379290,72.170360,2.057194e+06,3.818362e+07
std,41750.351353,1.542140,1.291206,730.883975,22.870219,35.697167,27.047692,1.279915e+06,8.154200e+06
min,7546.000000,-4.800000,-8.248530,953.426656,20.700000,10.633333,20.543000,3.978000e+03,2.054247e+07
25%,18413.200000,0.100000,1.021595,1880.503326,36.500000,26.433333,45.438000,9.493340e+05,3.281377e+07
50%,46764.000000,0.900000,1.465325,2542.306681,57.900000,47.900000,84.985000,1.824733e+06,4.143174e+07
75%,94390.100000,1.600000,2.111862,3157.752523,73.200000,85.666667,95.943000,2.884880e+06,4.376279e+07
max,143438.200000,6.600000,8.773860,3907.983317,100.400000,132.366667,107.388000,4.942294e+06,5.233012e+07


## Data Pre-Processing

Checking and removing legacy years which is missing majority of its data.

In [68]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 313 entries, 1947-06-30 to 2025-06-30
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sg_gdp          201 non-null    float64
 1   hk_gdp_growth   141 non-null    float64
 2   us_gdp_growth   312 non-null    float64
 3   sti_close       142 non-null    float64
 4   sg_cpi          257 non-null    float64
 5   iip             169 non-null    float64
 6   rsi             161 non-null    float64
 7   total_tourists  189 non-null    float64
 8   total_nodx      113 non-null    float64
dtypes: float64(9)
memory usage: 24.5 KB


In [61]:
print("\n### Initial Number of Missing Values per Column ###")
initial_missing_count = df_merged.isnull().sum()
print(initial_missing_count)


### Initial Number of Missing Values per Column ###
sg_gdp            112
hk_gdp_growth     172
us_gdp_growth       1
sti_close         171
sg_cpi             56
iip               144
rsi               152
total_tourists    124
total_nodx        200
dtype: int64


In [62]:
start_date = '1997-03-31'

df_processed = df_merged[df_merged.index >= start_date].copy()
df_processed

,sg_gdp,hk_gdp_growth,us_gdp_growth,sti_close,sg_cpi,iip,rsi,total_tourists,total_nodx
1997-03-31,41566.8,1.5,1.24560,2161.733317,62.7,31.366667,49.219,1898974.0,20542470.0
1997-06-30,42937.1,1.9,1.86747,2019.266683,63.0,32.700000,46.121,1881027.0,22565660.0
1997-09-30,43615.3,0.5,1.69035,1909.133301,63.6,35.966667,48.710,1824733.0,23597897.0
1997-12-31,43258.7,-2.7,1.18996,1592.166667,63.8,35.900000,46.530,1603523.0,24917677.0
1998-03-31,42144.3,-2.4,1.14732,1501.500000,63.4,33.466667,44.427,1523466.0,22750082.0
...,...,...,...,...,...,...,...,...,...
2024-06-30,138573.1,0.3,1.37173,3320.693359,99.9,116.833333,91.913,3993685.0,41660678.6
2024-09-30,142712.9,-0.1,1.23446,3494.719971,100.3,131.433333,94.232,4121205.0,45275088.3
2024-12-31,143438.2,0.9,1.18792,3695.256673,100.4,132.366667,100.985,4147741.0,44278329.8
2025-03-31,142508.4,1.9,0.85041,3907.983317,100.4,121.800000,97.089,4289701.0,43795208.6


In [63]:
print("\nNumber of Missing Values per Column:")
missing_count = df_processed.isnull().sum()
print(missing_count)
print("\n==========================")
df_processed[df_processed.isnull().any(axis=1)]


Number of Missing Values per Column:
sg_gdp            1
hk_gdp_growth     1
us_gdp_growth     1
sti_close         0
sg_cpi            1
iip               1
rsi               1
total_tourists    1
total_nodx        1
dtype: int64



,sg_gdp,hk_gdp_growth,us_gdp_growth,sti_close,sg_cpi,iip,rsi,total_tourists,total_nodx
2025-06-30,NaN,NaN,NaN,3863.560059,NaN,NaN,NaN,NaN,NaN


**Removing the Data Row with Missing Values**
* Not updated in official statistics yet as too recent

In [64]:
df_processed = df_processed.iloc[:-1].copy()
missing_count = df_processed.isnull().sum()
print(missing_count)

sg_gdp            0
hk_gdp_growth     0
us_gdp_growth     0
sti_close         0
sg_cpi            0
iip               0
rsi               0
total_tourists    0
total_nodx        0
dtype: int64


In [65]:
df_processed.to_csv("MergedData.csv")